<a href="https://colab.research.google.com/github/Amirhatamian/NLP/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install wikipedia-api


In [17]:
import nltk
from nltk.tokenize import sent_tokenize
import wikipediaapi

# Ensure required NLTK data is downloaded
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
def fetch_wikipedia_content(subject):
    # Initialize the Wikipedia API with the English language
    wiki_wiki = wikipediaapi.Wikipedia('english')

    # Fetch the Wikipedia page for the given subject
    page_py = wiki_wiki.page(subject)

    # Check if the page exists
    if not page_py.exists():
        print(f"Wikipedia page for '{subject}' does not exist.")
        return None

    # Return the text content of the page
    return page_py.text

# Example usage
subject = "Python (programming language)"
content = fetch_wikipedia_content(subject)
if content:
    print(content[:1000])  # Print the first 1000 characters of the content

Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.
Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.
Python consistently ranks as one of the most popular programming languages, and has gained widespread use in the machine learning community.

History
Python was invented in the late 1980s by Guido v

In [9]:
def save_content_to_file(content, filename):
    # Check if the content is not None
    if content is None:
        print("No content to save.")
        return

    # Write the content to a file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(content)

    print(f"Content saved to {filename}")

# Example usage
subject = "Python (programming language)"
content = fetch_wikipedia_content(subject)
if content:
    save_content_to_file(content, "python_programming_language.txt")


Content saved to python_programming_language.txt


In [10]:
def read_content_from_file(filename):
    try:
        # Open the file and read its content
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"File '{filename}' not found.")
        return None

# Example usage
filename = "python_programming_language.txt"
content = read_content_from_file(filename)
if content:
    print(content[:1000])  # Print the first 1000 characters of the content


Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.
Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.
Python consistently ranks as one of the most popular programming languages, and has gained widespread use in the machine learning community.

History
Python was invented in the late 1980s by Guido v

In [11]:
import nltk
from nltk.tokenize import sent_tokenize

# Download required NLTK data
nltk.download('punkt')

def summarize_text(text, max_sentences=5):
    """
    Summarize the text to fit within the context window.

    :param text: The text to be summarized.
    :param max_sentences: The maximum number of sentences for the summary.
    :return: The summarized text.
    """
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # If the text is already short enough, return it as is
    if len(sentences) <= max_sentences:
        return text

    # Simple summarization: take the first and last few sentences
    summary = sentences[:max_sentences//2] + sentences[-max_sentences//2:]

    return ' '.join(summary)

# Example usage
filename = "python_programming_language.txt"
content = read_content_from_file(filename)
if content:
    summary = summarize_text(content, max_sentences=5)
    print(summary)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. O'Reilly Media. ISBN 978-1-4920-5632-4. External links

Official website


In [12]:
def perform_summarization(content, output_filename, context_window_limit):
    """
    Summarize content to fit within the context window limit and save to file.

    :param content: The text content to be summarized.
    :param output_filename: The name of the file to save the summary.
    :param context_window_limit: The maximum word count for the context window.
    :return: The summarized text.
    """
    length = len(content.split())
    target_length = int(length * (context_window_limit / (context_window_limit + 4000)))

    summary_list = []
    start_index = 0
    content_words = content.split()

    while start_index < length:
        end_index = start_index + target_length
        slice_content = " ".join(content_words[start_index:end_index])
        summary_slice = summarize_text(slice_content, max_sentences=target_length//50)
        summary_list.append(summary_slice)
        start_index = end_index

    collated_summary = " ".join(summary_list)

    while len(collated_summary.split()) > context_window_limit:
        collated_summary = summarize_text(collated_summary, max_sentences=context_window_limit//50)

    with open(output_filename, "w", encoding="utf-8") as file:
        file.write(collated_summary)

    return collated_summary

# Example usage
filename = "python_programming_language.txt"
content = read_content_from_file(filename)
if content:
    summarized_content = perform_summarization(content, "summarized_python_programming_language.txt", 200)
    print(summarized_content)

Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. ISBN 978-1-4920-5632-4. External links Official website


In [13]:
# Main function
def main():
    # Take input from the user for the Wikipedia subjects
    subject1 = input("Enter the first subject: ")
    subject2 = input("Enter the second subject: ")

    # Fetch content from Wikipedia
    content1 = fetch_wikipedia_content(subject1)
    content2 = fetch_wikipedia_content(subject2)

    if content1 and content2:
        # Save content to input files
        save_content_to_file(content1, "input_text1.txt")
        save_content_to_file(content2, "input_text2.txt")

        # Summarize text files
        summarized_content1 = perform_summarization(content1, "summarized_text1.txt", 128)
        summarized_content2 = perform_summarization(content2, "summarized_text2.txt", 128)

        # Generate the query
        query = f"\nDocument 1 summary: {summarized_content1}\n\nDocument 2 summary: {summarized_content2}"

        print(query)

Content saved to input_text1.txt
Content saved to input_text2.txt


KeyboardInterrupt: 

In [ ]:
# Run the main function
if __name__ == "__main__":
    main()

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [14]:
def compute_similarity_matrix(sentences):
    """
    Compute the similarity matrix for a list of sentences.

    :param sentences: A list of sentences.
    :return: A similarity matrix.
    """
    # Transform sentences into a matrix of token counts
    vectorizer = CountVectorizer().fit_transform(sentences)

    # Compute cosine similarity between vectors
    similarity_matrix = cosine_similarity(vectorizer)

    return similarity_matrix

# Example usage
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "A fast brown fox leaps over a lazy dog.",
    "A quick dog jumps over a lazy fox.",
    "The lazy dog lies in the sun."
]

similarity_matrix = compute_similarity_matrix(sentences)
print(similarity_matrix)


[[1.         0.56980288 0.73854895 0.60302269]
 [0.56980288 1.         0.6172134  0.25197632]
 [0.73854895 0.6172134  1.         0.27216553]
 [0.60302269 0.25197632 0.27216553 1.        ]]


In [20]:
def summarize_text_cosine_similarity(text, target_length):
    sentences = sent_tokenize(text)

    if len(sentences) == 0:
        return ""

    # Compute cosine similarity matrix between sentences
    similarity_matrix = compute_similarity_matrix(sentences)

    length = 0
    summary_sentences = []
    selected_indices = set()

    while length < target_length and len(selected_indices) < len(sentences):
        if len(summary_sentences) == 0:
            # If summary is empty, start with the first sentence
            most_similar_index = 0
        else:
            # Find the sentence most similar to the existing summary
            summary_indices = [i for i in range(len(sentences)) if i in selected_indices]
            similarity_scores = similarity_matrix[summary_indices].sum(axis=0)
            similarity_scores[list(selected_indices)] = 0  # Avoid selecting already chosen sentences
            most_similar_index = np.argmax(similarity_scores)

        if most_similar_index in selected_indices:
            break

        # Add the selected sentence to the summary
        summary_sentences.append(sentences[most_similar_index])
        selected_indices.add(most_similar_index)

        # Update the length based on the added sentence
        length += len(sentences[most_similar_index].split())

    return ' '.join(summary_sentences).strip()

# Example usage
text = (
    "The quick brown fox jumps over the lazy dog. "
    "A fast brown fox leaps over a lazy dog. "
    "A quick dog jumps over a lazy fox. "
    "The lazy dog lies in the sun. "
    "The sun is shining brightly today. "
    "Foxes are known for their quick movements."
)
summary = summarize_text_cosine_similarity(text, 50)
print(summary)

The quick brown fox jumps over the lazy dog. A quick dog jumps over a lazy fox. A fast brown fox leaps over a lazy dog. The lazy dog lies in the sun. The sun is shining brightly today. Foxes are known for their quick movements.


In [21]:
def perform_summarization(content, output_filename, context_window_limit):
    length = len(content.split())
    target_length = int(length * (context_window_limit / (context_window_limit + 4000)))

    summary = summarize_text_cosine_similarity(content, target_length)

    with open(output_filename, "w", encoding="utf-8") as file:
        file.write(summary)

    return summary

# Example usage
filename = "python_programming_language.txt"
content = read_content_from_file(filename)
if content:
    summarized_content = perform_summarization(content, "summarized_python_programming_language.txt", 128)
    print(summarized_content)

Python is a high-level, general-purpose programming language. Design philosophy and features
Python is a multi-paradigm programming language. Syntax and semantics
Python is meant to be an easily readable language. It is also possible to cross-compile to other languages, but it either doesn't provide the full speed-up that might be expected, since Python is a very dynamic language, or a restricted subset of Python is compiled, and possibly semantics are slightly changed. Cross-compilers to other languages
There are several compilers/transpilers to high-level object languages, with either unrestricted Python, a restricted subset of Python, or a language similar to Python as the source language:

Brython, Transcrypt and Pyjs (latest release in 2012) compile Python to JavaScript. While Python 2.7 and older is officially unsupported, a different unofficial Python implementation, PyPy, continues to support Python 2, i.e. Examples of the use of this prefix in names of Python applications or l

In [22]:
def main():
    # Reading the content from input files
    contents_1 = read_content_from_file("input_text1.txt")
    contents_2 = read_content_from_file("input_text2.txt")

    if contents_1 and contents_2:
        # Perform summarization on both files
        perform_summarization(contents_1, "Second_summarized_text1.txt", 128)
        perform_summarization(contents_2, "Second_summarized_text2.txt", 128)

        # Generate the query with the summarized content
        query = f"\nDocument 1 summary: {read_content_from_file('Second_summarized_text1.txt')}\n" \
                f"\nDocument 2 summary: {read_content_from_file('Second_summarized_text2.txt')}"

        print(query)

if __name__ == "__main__":
    main()


Document 1 summary: NLP commonly refers to:

Natural language processing, a field of computer science and linguistics
Neuro-linguistic programming, a pseudoscientific method aimed at modifying human behavior
NLP or nLP may also refer to:

Computer programming
Natural-language programming, a programming paradigm

Libraries
National Library of Pakistan
National Library of Poland
National Library of the Philippines

Mathematics
Nonlinear programming, solving optimisation problems with nonlinear constraints

Medicine
No light perception, a diagnosis of severe blindness
Nasopharyngeal Laryngoscope
Lateral posterior nucleus of thalamus (nLP)

Political Parties
National Liberal Party (disambiguation)
National Liberation Party (disambiguation)
National Labour Party (disambiguation)
Natural Law Party, a trans-national union of political parties, with national branches in over 80 countries
Natural Law Party of Canada
Natural Law Party (Ireland)
Natural Law Party of Israel
Natural Law Party of N